# Создание различных ML моделей для определения спам сообщений

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pickle

In [2]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from nltk.corpus import stopwords

In [3]:
import sys
sys.path.append('../utils')  # Добавление папки ../utils в список путей поиска модулей.
from preprocessing import count_tags, count_links, count_emojis, count_newlines, count_whitespaces, count_newlines, replace_tags, replace_links, remove_emojis, preprocess_text

## Чтение обработанного датасета

In [4]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [5]:
df.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


## Подготовка выборок

In [6]:
texts = list(df['text_preprocessed'])  # Тексты
labels = list(df['label'])  # Метки (целевые значения)

In [7]:
additional_features = list(df[['emojis', 'newlines', 'whitespaces', 'links', 'tags']].values)  # Дополнительные числовые признаки

### Разделение на обучающую и тестовую выборки

In [8]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [9]:
X_train_num, X_test_num = train_test_split(
    additional_features, test_size=0.2, random_state=42, stratify=labels
)

### Подготовка стоп-слов

Загрузка английских и русских стоп-слов (`nltk`)

In [10]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
english_stopwords = set(ENGLISH_STOP_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\overklassniy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Объединение русских и английских стоп-слов

In [11]:
combined_stopwords = list(russian_stopwords.union(english_stopwords))

In [12]:
combined_stopwords

['five',
 'see',
 'нас',
 'у',
 'seemed',
 'over',
 'них',
 'ней',
 'become',
 'eight',
 'back',
 'herself',
 'один',
 'of',
 'meanwhile',
 'надо',
 'чуть',
 'fifty',
 'между',
 'although',
 'перед',
 'whereas',
 'most',
 'cry',
 'то',
 'well',
 'а',
 'я',
 'whether',
 'now',
 'please',
 'via',
 'anything',
 'rather',
 'move',
 'part',
 're',
 'everything',
 'became',
 'everywhere',
 'noone',
 'но',
 'разве',
 'though',
 'a',
 'them',
 'more',
 'below',
 'от',
 'either',
 'thereupon',
 'between',
 'her',
 'ну',
 'hers',
 'with',
 'они',
 'много',
 'nor',
 'than',
 'besides',
 'me',
 'might',
 'whenever',
 'inc',
 'из',
 'whereupon',
 'also',
 'им',
 'as',
 'мне',
 'теперь',
 'был',
 'name',
 'ours',
 'этого',
 'after',
 'due',
 'eleven',
 'ее',
 'all',
 'anyone',
 'top',
 'last',
 'moreover',
 'thereafter',
 'several',
 'yet',
 'keep',
 'before',
 'behind',
 'own',
 'any',
 'зачем',
 'next',
 'through',
 'is',
 'about',
 'very',
 'сейчас',
 'быть',
 'there',
 'two',
 'whatever',
 'enou

Проверка на наличие слов `tag` и `link` в списке стоп-слов.

In [13]:
'TAG' in combined_stopwords, 'tag' in combined_stopwords, 'LINK' in combined_stopwords, 'link' in combined_stopwords

(False, False, False, False)

In [14]:
len(combined_stopwords)

469

### Подготовка текстовых и числовых признаков

#### Преобразование текста в числовой формат (TF-IDF)

In [15]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=combined_stopwords,
    min_df=2,
    sublinear_tf=True
)

In [16]:
text_features_train = vectorizer.fit_transform(X_train_text)
text_features_test = vectorizer.transform(X_test_text)

#### Масштабирование числовых признаков

In [17]:
numerical_features_train = pd.DataFrame(X_train_num, columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags'])
numerical_features_test = pd.DataFrame(X_test_num, columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags'])

scaler = StandardScaler()
numerical_features_train_scaled = scaler.fit_transform(numerical_features_train)
numerical_features_test_scaled = scaler.transform(numerical_features_test)

#### Объединение текстовых и числовых признаков

In [18]:
X_train = hstack([text_features_train, numerical_features_train_scaled])
X_test = hstack([text_features_test, numerical_features_test_scaled])

#### Дополнительная синтетическая генерация

In [19]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Обучение моделей

### Logistic Regression

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE 

In [20]:
# 4. Обучение модели Logistic Regression
model_lr = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Logistic Regression + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [21]:
# 4. Обучение модели Logistic Regression
model_lr_sm = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr_sm.fit(X_train_resampled, y_train_resampled)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Random Forest

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

In [22]:
model_rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Random Forest + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [23]:
model_rf_sm = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf_sm.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Naive Bayes

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [24]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

Обучение Naive Bayes

In [25]:
model_nb = MultinomialNB()
model_nb.fit(X_train_nc, y_train)

MultinomialNB()

### Naive Bayes + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [26]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

In [27]:
smote = SMOTE(random_state=42)
X_train_resampled_nc, y_train_resampled_nc = smote.fit_resample(X_train_nc, y_train)

In [28]:
model_nb_sm = MultinomialNB()
model_nb_sm.fit(X_train_resampled_nc, y_train_resampled_nc)

MultinomialNB()

## Оценка качества моделей

Примеры тестовых сообщений

In [29]:
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!",
    "3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1",
    "Выиграл 345к дало x3450\n\nИграл тут: @jet_casino_ibot"
]

Функция для предсказания на новых данных

> Эта же функция находится в `../utils/preprocessing.py`.

In [30]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text: (str) - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_newlines(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

### Logistic Regression

In [31]:
y_pred = model_lr.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.99      0.95      4951
           1       0.99      0.93      0.96      6931

    accuracy                           0.95     11882
   macro avg       0.95      0.96      0.95     11882
weighted avg       0.96      0.95      0.95     11882

Confusion Matrix:
 [[4904   47]
 [ 519 6412]]


In [32]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9498622, 0.0501378]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.1567689, 0.8432311]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.6644723, 0.3355277]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.042467, 0.957533]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8528933, 0.1471067]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.185

### Logistic Regression + SMOTE

In [33]:
y_pred = model_lr_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95      4951
           1       0.99      0.93      0.96      6931

    accuracy                           0.95     11882
   macro avg       0.95      0.96      0.95     11882
weighted avg       0.96      0.95      0.95     11882

Confusion Matrix:
 [[4900   51]
 [ 509 6422]]


In [34]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9490555, 0.0509445]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.141632, 0.858368]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.668083, 0.331917]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.0370092, 0.9629908]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8602114, 0.1397886]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.13623

### Random Forest

In [35]:
y_pred = model_rf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.99      0.95      4951
           1       0.99      0.94      0.96      6931

    accuracy                           0.96     11882
   macro avg       0.95      0.96      0.96     11882
weighted avg       0.96      0.96      0.96     11882

Confusion Matrix:
 [[4877   74]
 [ 438 6493]]


In [36]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.92, 0.08]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.3366193, 0.6633807]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.93, 0.07]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.01, 0.99]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.953563, 0.046437]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.2934871, 0.7065129]

Сообщение: D

### Random Forest + SMOTE

In [37]:
y_pred = model_rf_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.99      0.95      4951
           1       0.99      0.94      0.96      6931

    accuracy                           0.96     11882
   macro avg       0.95      0.96      0.96     11882
weighted avg       0.96      0.96      0.96     11882

Confusion Matrix:
 [[4878   73]
 [ 449 6482]]





In [38]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.96, 0.04]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.3156212, 0.6843788]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.99, 0.01]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.04, 0.96]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9566368, 0.0433632]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.2138254, 0.7861746]

Сообщение:

### Naive Bayes

In [39]:
y_pred = model_nb.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67      4951
           1       1.00      0.29      0.45      6931

    accuracy                           0.59     11882
   macro avg       0.75      0.65      0.56     11882
weighted avg       0.79      0.59      0.54     11882

Confusion Matrix:
 [[4950    1]
 [4911 2020]]





In [40]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9995209, 0.0004791]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9961809, 0.0038191]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9978519, 0.0021481]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.9842836, 0.0157164]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9993462, 0.0006538]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.9

### Naive Bayes + SMOTE

In [41]:
y_pred = model_nb_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.51      1.00      0.67      4951
           1       1.00      0.30      0.47      6931

    accuracy                           0.59     11882
   macro avg       0.75      0.65      0.57     11882
weighted avg       0.79      0.59      0.55     11882

Confusion Matrix:
 [[4950    1]
 [4821 2110]]





In [42]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9995414, 0.0004586]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9967877, 0.0032123]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.998492, 0.001508]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.9837671, 0.0162329]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9995072, 0.0004928]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.997

## Сохранение результатов

In [43]:
with open('../models/vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

with open('../models/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Векторизатор и масштабатор сохранены!")

Векторизатор и масштабатор сохранены!


In [44]:
with open('../models/lr_model.pkl', 'wb') as lr_model_file:
    pickle.dump(model_lr, lr_model_file)

print("Модель логической регрессии без семплирования сохранена!")

Модель логической регрессии без семплирования сохранена!


In [45]:
with open('../models/lr_sm_model.pkl', 'wb') as lr_sm_model_file:
    pickle.dump(model_lr_sm, lr_sm_model_file)

print("Модель логической регрессии со семплированием сохранена!")

Модель логической регрессии со семплированием сохранена!


In [46]:
with open('../models/rf_model.pkl', 'wb') as rf_model_file:
    pickle.dump(model_rf, rf_model_file)

print("Модель случайного леса без семплирования сохранена!")

Модель случайного леса без семплирования сохранена!


In [47]:
with open('../models/rf_sm_model.pkl', 'wb') as rf_sm_model_file:
    pickle.dump(model_rf_sm, rf_sm_model_file)

print("Модель случайного леса со семплированием сохранена!")

Модель случайного леса со семплированием сохранена!


In [48]:
with open('../models/nb_model.pkl', 'wb') as nb_model_file:
    pickle.dump(model_nb, nb_model_file)

print("Модель Naive Bayes без семплирования сохранена!")

Модель Naive Bayes без семплирования сохранена!


In [49]:
with open('../models/nb_sm_model.pkl', 'wb') as nb_sm_model_file:
    pickle.dump(model_nb_sm, nb_sm_model_file)

print("Модель Naive Bayes со семплированием сохранена!")

Модель Naive Bayes со семплированием сохранена!
